In [ ]:
from astroCAST import preparation, analysis
from astroCAST.denoising import SubFrameGenerator, Network
from astroCAST.detection import Detector
from astroCAST.clustering import HdbScan, Distance, Linkage, Modules
from astroCAST.reduction import UMAP, CNN
from astroCAST.helper import Normalization
from astroCAST.analysis import Video, Events, Plotting

from pathlib import Path
import time
import logging
import h5py as h5
import matplotlib.pyplot as plt

folder = Path("/media/janrei1/data/method_testing/")
f = "img_100"

# Preprocessing

## Data conversion

## Motion Correction

## Denoising 

## Detrending

## Detection

# Analysis

In [ ]:
events = Events(folder.joinpath(f"{f}.h5").with_suffix(".ch0.roi"))
p = Plotting(events)

# events

## Exploration

## Clustering - unified

## Linkage

## Module

In [ ]:
t0=time.time()

### Extend
video = Video(folder.joinpath(f"{f}.h5"), h5_loc="dff/ch0/ch0")

events.filter({"dz":(2, 100)})
events.get_extended_events(video=video, extend=5, in_place=True)

### Normalization
# instructions = {
#     0: ["subtract", {"mode":"min"}],
#     1: ["divide", {"mode": "max_abs"}]
# }

# events.normalize(instructions)

distance = Distance()
modules = Modules(events)

print("correlation ...")
corr = distance.get_correlation(events, correlation_type="dtw")

print("graph ...")
nodes, edges, lct = modules.create_graph(corr, correlation_boundaries=(0.98, 1))
# modules.summarize_modules(nodes)

t1 = time.time()
print(f"dt: {t1-t0:.2f}s")

In [ ]:
import numpy as np
import napari

nodes["xy"] = nodes.apply(lambda x: np.array([x.x, x.y]), axis=1)
lut = dict(zip(nodes.trace_idx.tolist(), nodes.xy.tolist()))

L = np.array([
    [np.array(lut[e.source]), np.array(lut[e.target])-np.array(lut[e.source])] for _, e in edges.iterrows()
])
display(L.shape)


viewer.add_vectors(L, edge_width=1)

In [ ]:
v = events.show_event_map(video=folder.joinpath(f"{f}.h5"), h5_loc="dff/ch0/ch0")
v.add_vectors(L, edge_width=1)